# Loan Default Prediction Model

This notebook implements a machine learning pipeline to predict loan defaults. The model uses various features like credit score, income, loan amount, and other borrower characteristics to predict whether a loan will default.

## Data Processing Setup
The following cell imports required libraries and sets up data preprocessing steps including:
- Loading training and test datasets
- One-hot encoding categorical features 
- Splitting data into training and validation sets

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Load the dataset
data = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Define the categorical features
categorical_features = ['Education', 'EmploymentType', 'MaritalStatus', 'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner']

# Create a ColumnTransformer to apply OneHotEncoding to categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'  # Keep non-categorical columns as they are
)

# Fit and transform the training data, and transform the test data
X = preprocessor.fit_transform(data.drop(columns=['LoanID', 'Default']))  
X_tests = preprocessor.transform(test_df.drop(columns=['LoanID']))  
y = data['Default']  # Target variable
undropped_X_test = test_df  # Keep original test dataframe for submission

# Split data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def log_best_score(grid_search):
    best_score = grid_search.best_score_
    print(f"Best accuracy for model: {best_score:.4f}")


## Model Training and Hyperparameter Tuning

We implement three different models with grid search for hyperparameter optimization:

1. Decision Tree Classifier (currently commented out)
2. XGBoost Classifier (currently commented out)
3. Gradient Boosting Classifier (active)

Each model's hyperparameter grid focuses on key parameters that affect model complexity and learning rate.

In [3]:
# Define hyperparameter grids for each model
dt_params = {
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}

xgb_params = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 6, 10],
    'n_estimators': [50, 100, 200]
}

gb_params = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7]
}

# Train and tune Gradient Boosting model
gb_grid_search = GridSearchCV(GradientBoostingClassifier(random_state=42), 
                             gb_params, 
                             scoring='accuracy', 
                             cv=5, 
                             n_jobs=-1, 
                             verbose=True)
gb_grid_search.fit(X_train, y_train)
log_best_score(gb_grid_search)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best accuracy for model: 0.8862


## Submission Generation

The following cells define a utility function to generate submission files and preview the test data structure.

In [4]:
def generate_submission(predictions, df):
    """Generate a submission file with loan IDs and predicted defaults"""
    submission = pd.DataFrame({'LoanID': df['LoanID'], 'Default': predictions})
    submission.to_csv("LendOrLose_submission.csv", index=False)

## Final Prediction and Submission

Use the best Gradient Boosting model to make predictions on the test set and generate submission file.

In [7]:
gb_tuned = gb_grid_search.best_estimator_
gb_prediction = gb_tuned.predict(X=X_tests)
generate_submission(gb_prediction, undropped_X_test)